In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import re

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [ ]:
def reduce_memory_usage(df:pd.DataFrame) -> pd.DataFrame:
        numerics = ['int8', 'int16', 'int32', 'int64', 'float8','float16', 'float32', 'float64']
        start_mem = df.memory_usage().sum() / 1024**2
        for col in tqdm(df.columns):
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if (c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max) or (c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max):
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if (c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max) or (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        end_mem = df.memory_usage().sum() / 1024**2
        print(f'Mem. usage decreased to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
        return df

In [ ]:
demo_test_df = reduce_memory_usage(pd.read_csv('testdemographics.csv'))
#demo_train_df=pd.read_csv('traindemographics.csv')
prev_test_df = reduce_memory_usage(pd.read_csv('testprevloans.csv',
                                               parse_dates = ["approveddate", "creationdate", "closeddate", "firstduedate", "firstrepaiddate"]))
#prev_train_df=  pd.read_csv('trainprevloans.csv')
#perf_train_df=pd.read_csv('trainperf.csv')
perf_test_df = reduce_memory_usage(pd.read_csv(os.path.join('testperf.csv'),
                                             parse_dates = ["approveddate", "creationdate"]))

100%|██████████| 9/9 [00:00<00:00, 2126.93it/s]


Mem. usage decreased to  0.09 Mb (11.1% reduction)


100%|██████████| 12/12 [00:00<00:00, 1481.13it/s]


Mem. usage decreased to  0.41 Mb (25.0% reduction)


100%|██████████| 9/9 [00:00<00:00, 1463.02it/s]

Mem. usage decreased to  0.07 Mb (33.3% reduction)


In [ ]:
# check for duplicates in demographic dataset..
print(f"No. of Duplicate Records: {demo_test_df.duplicated(subset = ['customerid'], keep = 'last').sum()}")

No. of Duplicate Records: 3


In [ ]:
# get the duplicated records in the demographic dataset, while keeping the last records....
demo_test_df[demo_test_df.duplicated(subset = ['customerid'], keep = 'last') == True]

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
248,8a858e885b68546c015b7238f22027ba,1975-06-18 00:00:00.000000,Savings,3.353123,6.587837,First Bank,NaN,Permanent,NaN
497,8a8588b75438f49901544510041e0951,1980-01-16 00:00:00.000000,Other,3.342808,6.542411,Zenith Bank,NaN,Permanent,Graduate
1252,8a858f2e5c699f3a015c6e10b65607c9,1978-06-20 00:00:00.000000,Savings,3.325513,6.500937,GT Bank,NaN,Permanent,NaN


In [ ]:
# remove duplicated rows..
demo_test_df.drop_duplicates(subset = ['customerid'], keep = 'last', inplace = True)

In [ ]:
# check duplicate rows in previous performance dataset..
# check for duplicate customer ids...
print(f"No. of Duplicate Records: {prev_test_df.duplicated(subset = ['customerid'], keep = 'last').sum()}")

No. of Duplicate Records: 4465


In [ ]:
# check for full row duplicate...
# no full row duplicates....
print(f"No. of Duplicate Records: {prev_test_df.duplicated(subset = ['customerid', 'systemloanid'], keep = 'last').sum()}")

No. of Duplicate Records: 0


In [ ]:
# adding features to previous performance dataset..
# additional fees...
prev_test_df["fees"] = prev_test_df["totaldue"] - prev_test_df["loanamount"]
# number of days between loan approval date and closed date to see the difference between it and termdays and if it exceeds termdays...
prev_test_df["process_time"] = (prev_test_df['closeddate'] - prev_test_df['approveddate']).dt.days
prev_test_df["exceeds_termdays"] = prev_test_df.apply(lambda row: 1 if  row["process_time"] > row["termdays"] else 0, axis = 1)
prev_test_df["termdays_actual_diff"] = prev_test_df['termdays'] - prev_test_df['process_time']
# firstduedate: first planned payment date...
# firstrepaiddate : first actual payment date....
# number of days between planned and actual dates may be an indicator..
prev_test_df["after_planned"] = prev_test_df.apply(lambda row: 1 if  row["firstrepaiddate"] > row["firstduedate"] else 0, axis = 1)
prev_test_df["planned_actual_diff"] = (prev_test_df['firstduedate'] - prev_test_df['firstrepaiddate']).dt.days
# difference between creation and approval date..
prev_test_df["approval_creation_diff"] = (prev_test_df['approveddate'] - prev_test_df['creationdate']).dt.seconds / 60.0**2

In [ ]:
#Now we need to aggregate the previous performance dataset to make it efficient for us when joining the datasets..
# number of previous loans...
agg_prev_test_df = pd.DataFrame(
    prev_test_df.groupby("customerid", as_index = False).agg({
        'systemloanid':'count',
        'termdays': ['mean', 'min', 'max'],
        'loanamount': ['mean', 'min', 'max'],
        'fees': ['mean', 'min', 'max'],
        'exceeds_termdays': 'sum',
        'termdays_actual_diff': 'mean',
        'after_planned': 'sum',
        'planned_actual_diff': 'mean',
        'approval_creation_diff': 'mean'
    })
)
agg_prev_test_df.columns = ["customerid", "loan_cnt", "termdays_mean", "termdays_min", "termdays_max",
                             "loan_amt_mean", "loan_amt_min", "loan_amt_max", "fees_mean", "fees_min", "fees_max",
                            "exceeds_termdays_sum", "avg_termdays_actual_diff", "after_planned_sum", "avg_planned_actual_diff",
                            "avg_approval_creation_diff"]
agg_prev_test_df.head()

,customerid,loan_cnt,termdays_mean,termdays_min,termdays_max,loan_amt_mean,loan_amt_min,loan_amt_max,fees_mean,fees_min,fees_max,exceeds_termdays_sum,avg_termdays_actual_diff,after_planned_sum,avg_planned_actual_diff,avg_approval_creation_diff
0,8a28afc7474813a40147639ec637156b,10,31.5,15,60,17800.000000,8000.0,30000.0,3290.000000,1500.0,6000.0,1,6.000000,4,2.300000,1.003611
1,8a3735d5518aba7301518ac34413010d,6,27.5,15,30,11666.666992,10000.0,20000.0,3000.000000,1500.0,4500.0,1,13.333333,1,11.666667,1.005556
2,8a76e7d443e6e97c0143ed099d102b1d,2,15.0,15,15,10000.000000,10000.0,10000.0,1500.000000,1500.0,1500.0,0,1.500000,0,1.000000,1.004583
3,8a818823525dceef01525deda2480384,9,40.0,30,60,18888.888672,5000.0,40000.0,4441.666504,1500.0,8000.0,3,0.777778,8,-1.111111,2.668981
4,8a818926522ea5ef01523aff15c37482,2,30.0,30,30,10000.000000,10000.0,10000.0,3000.000000,3000.0,3000.0,2,-10.500000,2,-9.500000,1.008750


In [ ]:
agg_prev_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442 entries, 0 to 1441
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  1442 non-null   object 
 1   loan_cnt                    1442 non-null   int64  
 2   termdays_mean               1442 non-null   float64
 3   termdays_min                1442 non-null   int16  
 4   termdays_max                1442 non-null   int16  
 5   loan_amt_mean               1442 non-null   float32
 6   loan_amt_min                1442 non-null   float32
 7   loan_amt_max                1442 non-null   float32
 8   fees_mean                   1442 non-null   float32
 9   fees_min                    1442 non-null   float32
 10  fees_max                    1442 non-null   float32
 11  exceeds_termdays_sum        1442 non-null   int64  
 12  avg_termdays_actual_diff    1442 non-null   float64
 13  after_planned_sum           1442 

In [ ]:
# no duplicate rows in performance dataset, perfect !!!!!!!
print(f"No. of Duplicate Records: {perf_test_df.duplicated(subset = ['customerid'], keep = 'last').sum()}")

No. of Duplicate Records: 0


In [ ]:
# to start working on the dataset and analyze them effectively
# let's join them together..
# left join demographics and previous performance datasets on performance dataset..
final_test_df = perf_test_df.merge(agg_prev_test_df,
                                    on = "customerid",
                                    how = "left",
                                    suffixes=('_perf', '_prev'))
final_test_df = final_test_df.merge(demo_test_df,
                                      on = "customerid",
                                      how = "left",
                                      suffixes=('', '_demo'))
final_test_df.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,loan_cnt,termdays_mean,termdays_min,termdays_max,loan_amt_mean,loan_amt_min,loan_amt_max,fees_mean,fees_min,fees_max,exceeds_termdays_sum,avg_termdays_actual_diff,after_planned_sum,avg_planned_actual_diff,avg_approval_creation_diff,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858899538ddb8e015390510b321f08,301998974,4,40:48.0,39:35.0,10000,12250.0,30,NaN,3.0,25.000000,15.0,30.0,8666.666992,6000.0,10000.0,2300.000000,900.0,3000.0,1.0,-3.333333,2.0,-4.333333,1.001944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8a858959537a097401537a4e316e25f7,301963615,10,43:40.0,42:34.0,40000,44000.0,30,NaN,9.0,31.666667,15.0,60.0,23888.888672,5000.0,40000.0,3711.111084,1500.0,6800.0,1.0,7.555556,2.0,2.888889,1.002191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,15:11.0,15:04.0,20000,24500.0,30,NaN,5.0,27.000000,15.0,30.0,13000.000000,10000.0,20000.0,2935.000000,1500.0,3800.0,3.0,-4.200000,3.0,-4.600000,1.005389,1981-09-05 00:00:00.000000,Savings,3.227945,6.586668,UBA,NaN,Permanent,NaN
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,00:54.0,00:49.0,30000,34500.0,30,NaN,7.0,17.142857,15.0,30.0,17142.857422,10000.0,30000.0,2200.000000,1500.0,4500.0,0.0,3.714286,0.0,2.285714,1.006667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a858e225a28c713015a30db5c48383d,301959177,4,04:33.0,04:27.0,20000,24500.0,30,NaN,3.0,25.000000,15.0,30.0,10000.000000,10000.0,10000.0,2500.000000,1500.0,3000.0,0.0,3.333333,1.0,2.333333,1.002870,1975-08-25 00:00:00.000000,Savings,5.248368,13.059864,UBA,NaN,Permanent,NaN


In [ ]:
# check for full-row duplicates..
print(f"No. of Full row duplicate records: {final_test_df.duplicated().sum()}")

No. of Full row duplicate records: 0


In [ ]:
final_test_df["level_of_education_clients"].value_counts(normalize = True, dropna = False)

NaN              0.967586
Graduate         0.022759
Secondary        0.004828
Post-Graduate    0.004138
Primary          0.000690
Name: level_of_education_clients, dtype: float64

In [ ]:
final_test_df["employment_status_clients"].value_counts(normalize = True, dropna = False)

NaN              0.771034
Permanent        0.193793
Self-Employed    0.021379
Student          0.011034
Unemployed       0.002069
Retired          0.000690
Name: employment_status_clients, dtype: float64

In [ ]:
final_test_df["bank_name_clients"].value_counts(normalize = True, dropna = False)

NaN              0.734483
GT Bank          0.094483
First Bank       0.039310
Access Bank      0.024138
UBA              0.023448
Diamond Bank     0.019310
Zenith Bank      0.017241
FCMB             0.012414
Stanbic IBTC     0.008276
EcoBank          0.006897
Skye Bank        0.006207
Fidelity Bank    0.004828
Union Bank       0.003448
Sterling Bank    0.002069
Wema Bank        0.002069
Heritage Bank    0.000690
Unity Bank       0.000690
Name: bank_name_clients, dtype: float64

In [ ]:
final_test_df["bank_account_type"].value_counts(normalize = True, dropna = False)

NaN        0.734483
Savings    0.213103
Other      0.048276
Current    0.004138
Name: bank_account_type, dtype: float64

In [ ]:
# data cleaning steps...
## 1- referredby_perf, referredby_prev if missing then customers aren't referred, replace missing by no-referral
## 2- 99.4% of 'bank_branch_clients' values are missing then we can remove it safely..
## 3- replace missing values in level_of_education_clients by no-info
## 4- replace missing values in employment_status_clients by no-info
final_test_df.drop("bank_branch_clients", axis = 1, inplace = True)

In [ ]:
final_test_df['approveddate']#.info()

0       40:48.0
1       43:40.0
2       15:11.0
3       00:54.0
4       04:33.0
         ...   
1445    47:02.0
1446    00:45.0
1447    09:40.0
1448    47:54.0
1449    12:09.0
Name: approveddate, Length: 1450, dtype: object

In [ ]:
final_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1450 entries, 0 to 1449
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  1450 non-null   object 
 1   systemloanid                1450 non-null   int32  
 2   loannumber                  1450 non-null   int16  
 3   approveddate                1450 non-null   object 
 4   creationdate                1450 non-null   object 
 5   loanamount                  1450 non-null   int32  
 6   totaldue                    1450 non-null   float32
 7   termdays                    1450 non-null   int16  
 8   referredby                  184 non-null    object 
 9   loan_cnt                    1442 non-null   float64
 10  termdays_mean               1442 non-null   float64
 11  termdays_min                1442 non-null   float64
 12  termdays_max                1442 non-null   float64
 13  loan_amt_mean               1442 

In [ ]:
final_test_df.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,loan_cnt,termdays_mean,termdays_min,termdays_max,loan_amt_mean,loan_amt_min,loan_amt_max,fees_mean,fees_min,fees_max,exceeds_termdays_sum,avg_termdays_actual_diff,after_planned_sum,avg_planned_actual_diff,avg_approval_creation_diff,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,level_of_education_clients
0,8a858899538ddb8e015390510b321f08,301998974,4,40:48.0,39:35.0,10000,12250.0,30,NaN,3.0,25.000000,15.0,30.0,8666.666992,6000.0,10000.0,2300.000000,900.0,3000.0,1.0,-3.333333,2.0,-4.333333,1.001944,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8a858959537a097401537a4e316e25f7,301963615,10,43:40.0,42:34.0,40000,44000.0,30,NaN,9.0,31.666667,15.0,60.0,23888.888672,5000.0,40000.0,3711.111084,1500.0,6800.0,1.0,7.555556,2.0,2.888889,1.002191,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,15:11.0,15:04.0,20000,24500.0,30,NaN,5.0,27.000000,15.0,30.0,13000.000000,10000.0,20000.0,2935.000000,1500.0,3800.0,3.0,-4.200000,3.0,-4.600000,1.005389,1981-09-05 00:00:00.000000,Savings,3.227945,6.586668,UBA,Permanent,NaN
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,00:54.0,00:49.0,30000,34500.0,30,NaN,7.0,17.142857,15.0,30.0,17142.857422,10000.0,30000.0,2200.000000,1500.0,4500.0,0.0,3.714286,0.0,2.285714,1.006667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a858e225a28c713015a30db5c48383d,301959177,4,04:33.0,04:27.0,20000,24500.0,30,NaN,3.0,25.000000,15.0,30.0,10000.000000,10000.0,10000.0,2500.000000,1500.0,3000.0,0.0,3.333333,1.0,2.333333,1.002870,1975-08-25 00:00:00.000000,Savings,5.248368,13.059864,UBA,Permanent,NaN


In [ ]:
import pandas as pd

# Assuming final_test_df is your DataFrame

# Define a specific date and hour for combining with time
specific_date = '2023-08-01'
specific_hour = '00'  # Add the hour you want

# Convert 'approveddate' column to datetime by combining with specific date and hour
final_test_df['approveddate'] = pd.to_datetime(specific_date + ' ' + specific_hour + ':' + final_test_df['approveddate'], format='%Y-%m-%d %H:%M:%S.%f')

# Convert 'creationdate' column to datetime by combining with specific date and hour
final_test_df['creationdate'] = pd.to_datetime(specific_date + ' ' + specific_hour + ':' + final_test_df['creationdate'], format='%Y-%m-%d %H:%M:%S.%f')

# Calculate time difference in hours
final_test_df["approval_creation_diff"] = (final_test_df['approveddate'] - final_test_df['creationdate']).dt.seconds / 3600.0

# Drop the original date columns
final_test_df.drop(['approveddate', 'creationdate'], axis=1, inplace=True)


In [ ]:
final_test_df.to_csv("final_test.csv", index = False)

In [ ]:
final_test_df

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,referredby,loan_cnt,termdays_mean,termdays_min,termdays_max,loan_amt_mean,loan_amt_min,loan_amt_max,fees_mean,fees_min,fees_max,exceeds_termdays_sum,avg_termdays_actual_diff,after_planned_sum,avg_planned_actual_diff,avg_approval_creation_diff,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,level_of_education_clients,approval_creation_diff
0,8a858899538ddb8e015390510b321f08,301998974,4,10000,12250.0,30,NaN,3.0,25.000000,15.0,30.0,8666.666992,6000.0,10000.0,2300.000000,900.0,3000.0,1.0,-3.333333,2.0,-4.333333,1.001944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.020278
1,8a858959537a097401537a4e316e25f7,301963615,10,40000,44000.0,30,NaN,9.0,31.666667,15.0,60.0,23888.888672,5000.0,40000.0,3711.111084,1500.0,6800.0,1.0,7.555556,2.0,2.888889,1.002191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018333
2,8a8589c253ace09b0153af6ba58f1f31,301982236,6,20000,24500.0,30,NaN,5.0,27.000000,15.0,30.0,13000.000000,10000.0,20000.0,2935.000000,1500.0,3800.0,3.0,-4.200000,3.0,-4.600000,1.005389,1981-09-05 00:00:00.000000,Savings,3.227945,6.586668,UBA,Permanent,NaN,0.001944
3,8a858e095aae82b7015aae86ca1e030b,301971730,8,30000,34500.0,30,NaN,7.0,17.142857,15.0,30.0,17142.857422,10000.0,30000.0,2200.000000,1500.0,4500.0,0.0,3.714286,0.0,2.285714,1.006667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001389
4,8a858e225a28c713015a30db5c48383d,301959177,4,20000,24500.0,30,NaN,3.0,25.000000,15.0,30.0,10000.000000,10000.0,10000.0,2500.000000,1500.0,3000.0,0.0,3.333333,1.0,2.333333,1.002870,1975-08-25 00:00:00.000000,Savings,5.248368,13.059864,UBA,Permanent,NaN,0.001667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,301991141,3,10000,11500.0,15,NaN,2.0,15.000000,15.0,15.0,10000.000000,10000.0,10000.0,1500.000000,1500.0,1500.0,0.0,5.000000,0.0,5.000000,1.010417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002500
1446,8a858fcb5b00cc54015b0253ced26a5f,301963018,3,10000,13000.0,30,NaN,2.0,15.000000,15.0,15.0,10000.000000,10000.0,10000.0,1500.000000,1500.0,1500.0,0.0,1.500000,1.0,0.000000,1.010278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.944444
1447,8a858fde56eb02280156f59b976d46c0,301994653,9,40000,48000.0,60,NaN,8.0,37.500000,30.0,60.0,18125.000000,10000.0,30000.0,3950.000000,2250.0,6800.0,2.0,2.875000,1.0,3.500000,1.005243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001944
1448,8a858e10570f2d65015717fcfec44996,301961483,14,60000,68100.0,90,NaN,13.0,54.230769,15.0,90.0,30384.615234,10000.0,50000.0,5700.000000,1500.0,7600.0,0.0,33.153846,2.0,7.384615,1.002778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018056


In [ ]:
final_test_df.drop(columns=['birthdate','customerid', 'systemloanid'])
final_test_df["referredby"] = final_test_df["referredby"].apply(lambda value: 'yes' if pd.notnull(value) else 'No')


In [ ]:
import pickle
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Load the saved model
with open('xgb_classifier.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


# Make predictions using the loaded model
preds = loaded_model.predict(final_test_df)




In [ ]:
len(preds[preds == 0]) / len(preds)

0.05310344827586207

In [ ]:
from sklearn.linear_model import LogisticRegression

# Load the saved model
with open('logreg.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


# Make predictions using the loaded model
preds = loaded_model.predict(final_test_df)


In [ ]:
sub_df = pd.concat([perf_test_df['customerid'], pd.Series(preds)], axis = 1)
sub_df.columns = ['customerid', 'Good_Bad_flag']
sub_df

,customerid,Good_Bad_flag
0,8a858899538ddb8e015390510b321f08,1
1,8a858959537a097401537a4e316e25f7,1
2,8a8589c253ace09b0153af6ba58f1f31,1
3,8a858e095aae82b7015aae86ca1e030b,1
4,8a858e225a28c713015a30db5c48383d,1
...,...,...
1445,8a858fb45bb59c21015bb88a191f58f2,1
1446,8a858fcb5b00cc54015b0253ced26a5f,0
1447,8a858fde56eb02280156f59b976d46c0,1
1448,8a858e10570f2d65015717fcfec44996,1


In [ ]:
sub_df.to_csv('submission_df.csv',index=False)